In [1]:
import yaml
import os

with open('data.yaml', 'r') as file:
    data = yaml.safe_load(file)

print("Train path:", os.path.exists(data['train']))
print("Validation path:", os.path.exists(data['val']))
print("Test path:", os.path.exists(data['test']))

def count_images(directory):
    supported_formats = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')
    return len([name for name in os.listdir(directory) if name.lower().endswith(supported_formats)])

print(f" The number of images in training set: {count_images(data['train'])}")
print(f" The number of images in validation set: {count_images(data['val'])}")
print(f" The number of images in test set: {count_images(data['test'])}")


Train path: True
Validation path: True
Test path: True
 The number of images in training set: 1228
 The number of images in validation set: 168
 The number of images in test set: 84


In [2]:
from ultralytics import YOLO
from pathlib import Path

absolute_from_relative_path = Path('data.yaml').resolve()

model = YOLO('yolov8n.pt')  # Load a pretrained model (recommended for training)
results = model.train(data=absolute_from_relative_path, epochs=40, imgsz=480)


New https://pypi.org/project/ultralytics/8.2.26 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.227 🚀 Python-3.9.18 torch-1.13.1 CPU (Apple M1)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/alessiacolumban/Desktop/2023-2024-projectone-ctai-AlessCol7/PillPoint/AI/PillPoint.v1i.yolov8/data.yaml, epochs=40, patience=50, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, a

Fontconfig warning: ignoring UTF-8: not a valid region tag
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /Users/alessiacolumban/Desktop/2023-2024-projectone-ctai-AlessCol7/PillPoint/AI/PillPoint.v1i.yolov8/train/labels.cache... 1228 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1228/1228 [00:00<?, ?it/s]
val: Scanning /Users/alessiacolumban/Desktop/2023-2024-projectone-ctai-AlessCol7/PillPoint/AI/PillPoint.v1i.yolov8/valid/labels.cache... 168 images, 0 backgrounds, 0 corrupt: 100%|██████████| 168/168 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 480 train, 480 val
Using 0 dataloader workers
Logging results to runs/detect/train3
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/40         0G      1.578      2.189      1.541         77        480:  99%|█████████▊| 76/77 [04:41<00:03,  3.71s/it]


KeyboardInterrupt: 

In [3]:
import cv2
from ultralytics import YOLO
from PIL import Image, ImageTk
import tkinter as tk

# Load the trained YOLOv8 model
try:
    model = YOLO('runs/detect/train2/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['Hands', 'Pills', 'PillBoxes']

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results):
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        conf = confs[i]
        cls = int(classes[i])
        label = class_names[cls]
        if label == 'Hands':
            color = (0, 255, 0)  # Green
        elif label == 'Pill':
            color = (255, 0, 0)  # Red
        else:
            color = (0, 0, 255)  # Blue
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return image

# Initialize the webcam
cap = cv2.VideoCapture(0)  # Use 0 for the default webcam, or specify the webcam index

def update_frame():
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame from webcam")
        return

    # Perform inference with adjusted NMS settings
    try:
        results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return

    # Annotate the frame with detection results
    frame = draw_bounding_boxes(frame, results)

    # Convert the frame to an ImageTk object
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    imgtk = ImageTk.PhotoImage(image=img)

    # Update the label with the new image
    lbl.imgtk = imgtk  # Keep a reference to the image to prevent garbage collection
    lbl.configure(image=imgtk)

    # Schedule the next update
    lbl.after(10, update_frame)

# Create the main window
root = tk.Tk()
root.title("Webcam Detection")

# Create a label to display the video feed
lbl = tk.Label(root)
lbl.pack()

# Start the Tkinter event loop and the video feed
root.after(0, update_frame)
root.mainloop()

# Release the webcam
cap.release()


[ WARN:0@329.346] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created



0: 288x480 (no detections), 76.0ms
Speed: 3.1ms preprocess, 76.0ms inference, 0.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 Hands, 46.3ms
Speed: 1.1ms preprocess, 46.3ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 Hands, 47.6ms
Speed: 1.1ms preprocess, 47.6ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 45.0ms
Speed: 1.6ms preprocess, 45.0ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 39.6ms
Speed: 2.0ms preprocess, 39.6ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 38.7ms
Speed: 0.9ms preprocess, 38.7ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 44.0ms
Speed: 1.3ms preprocess, 44.0ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 Hands, 39.5ms
Speed: 1.2ms preprocess, 39.5ms inference, 0.6ms postproc

: 

In [1]:
import socket
import threading
import time
import sys


server_address = ('192.168.168.167', 1441)  # Connect to RPi (or other server) on ip ... and port ... (the port is set in server.py)
# the ip address can also be the WiFi ip of your RPi, but this can change. You can print your WiFi IP on your LCD? (if needed)


# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event() # see: https://docs.python.org/3/library/threading.html#event-objects


def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # create a socket instance
    client_socket.connect(server_address) # connect to specified server
    print("Connected to server")

    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    sock.settimeout(1)  # Set a timeout on the socket so when can check shutdown_flag.is_set in the loop, instead of blocking
    counter = 0 # count the incoming messages, part of demo
    try:
        while not shutdown_flag.is_set(): # as long as ctrl+c is not pressed
            try:
                data = sock.recv(1024) # try to receive 1024 bytes of data (maximum amount; can be less)
                if not data: # when no data is received, try again (and shutdown flag is checked again)
                    break
                print("Received from server:", data.decode()) # print the received data, or do something with it
                counter += 1 # up the count by 1
                response = "{} message(s) received".format(counter) # create a response string
                sock.sendall(response.encode()) # encode and send the data
            except socket.timeout: # when no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

def main():
    global client_socket, receive_thread

    setup_socket_client()

    if client_socket is None:
        print("Not connected, is server running on {}:{}?".format(server_address[0], server_address[1]))
        sys.exit()
    
    # send "hello I'm connected" message
    client_socket.sendall("Hello from AI / notebook".encode()) # send a "connected" message from client > server
        

    try:
        while True: # random loop for other things
            time.sleep(6)
            print("doing other things...")
    except KeyboardInterrupt:
        print("Client disconnecting...")
        shutdown_flag.set()
    finally:
        client_socket.close()
        receive_thread.join()
        print("Client stopped gracefully")

if __name__ == "__main__":
    main()


Connected to server
doing other things...
Received from server: Hello from RPi loop
doing other things...
doing other things...
Received from server: Hello from RPi loop
doing other things...
Received from server: Hello from RPi loop
doing other things...
doing other things...
Received from server: Hello from RPi loop
doing other things...
doing other things...
Received from server: Hello from RPi loop
doing other things...
Received from server: Hello from RPi loop
doing other things...
doing other things...
Received from server: Hello from RPi loop
doing other things...
doing other things...
Received from server: Hello from RPi loop
doing other things...
Received from server: Hello from RPi loop
doing other things...
doing other things...
Received from server: Hello from RPi loop
doing other things...
doing other things...
Received from server: Hello from RPi loop
doing other things...
Received from server: Hello from RPi loop
doing other things...
doing other things...
Received from 

In [2]:
import cv2
from ultralytics import YOLO
from PIL import Image, ImageTk
import tkinter as tk
import socket
import threading
import time


# Load the trained YOLOv8 model
try:
    model = YOLO('runs/detect/train2/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['Hands', 'Pills', 'PillBoxes']

# Server address and port for Raspberry Pi
server_address = ('192.168.168.167', 1442)  # Change this to your Raspberry Pi IP and port

# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event()
last_notification_time = 0
notification_interval = 1

def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket instance
    client_socket.connect(server_address)  # Connect to specified server
    print("Connected to server")

    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    sock.settimeout(1)  # Set a timeout on the socket so we can check shutdown_flag.is_set in the loop, instead of blocking
    try:
        while not shutdown_flag.is_set():  # As long as ctrl+c is not pressed
            try:
                data = sock.recv(1024)  # Try to receive 1024 bytes of data (maximum amount; can be less)
                if not data:  # When no data is received, try again (and shutdown flag is checked again)
                    break
                print("Received from server:", data.decode())  # Print the received data, or do something with it
            except socket.timeout:  # When no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results):
    global last_notification_time
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        conf = confs[i]
        cls = int(classes[i])
        label = class_names[cls]
        if label == 'Hands':
            color = (0, 255, 0)  # Green
        elif label == 'Pills':
            color = (255, 0, 0)  # Red
            # Send notification to Raspberry Pi when wrong pill is detected
            current_time = time.time()
            if current_time - last_notification_time >= notification_interval:
                notify_raspberry_pi("Wrong pill detected")
                last_notification_time = current_time
        else:
            color = (0, 0, 255)  # Blue
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return image

# Function to notify Raspberry Pi
def notify_raspberry_pi(message):
    try:
        client_socket.sendall(message.encode())
        print(f"Sent to Raspberry Pi: {message}")
    except Exception as e:
        print(f"Error sending message to Raspberry Pi: {e}")

# Initialize the webcam
cap = cv2.VideoCapture(0)  # Use 0 for the default webcam, or specify the webcam index

def update_frame():
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame from webcam")
        return

    # Perform inference with adjusted NMS settings
    try:
        results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return

    # Annotate the frame with detection results
    frame = draw_bounding_boxes(frame, results)

    # Convert the frame to an ImageTk object
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    imgtk = ImageTk.PhotoImage(image=img)

    # Update the label with the new image
    lbl.imgtk = imgtk  # Keep a reference to the image to prevent garbage collection
    lbl.configure(image=imgtk)

    # Schedule the next update
    lbl.after(10, update_frame)

# Create the main window
root = tk.Tk()
root.title("Webcam Detection")

# Create a label to display the video feed
lbl = tk.Label(root)
lbl.pack()

# Setup the socket client and start the update loop
setup_socket_client()
root.after(0, update_frame)
root.mainloop()

# Cleanup
cap.release()
shutdown_flag.set()
if client_socket:
    client_socket.close()
if receive_thread:
    receive_thread.join()


[ WARN:0@141.347] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Connected to server

0: 288x480 (no detections), 53.1ms
Speed: 3.2ms preprocess, 53.1ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 70.3ms
Speed: 2.0ms preprocess, 70.3ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 48.9ms
Speed: 1.4ms preprocess, 48.9ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 47.1ms
Speed: 1.2ms preprocess, 47.1ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 40.8ms
Speed: 1.1ms preprocess, 40.8ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 44.2ms
Speed: 1.3ms preprocess, 44.2ms inference, 0.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 47.5ms
Speed: 1.4ms preprocess, 47.5ms inference, 6.6ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 46.4ms
Speed: 1.2ms preprocess, 

: 